<a href="https://colab.research.google.com/github/happyfranc/ml_project/blob/main/%08DL_project1_v1_%EB%B0%95%EC%9D%B4%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

생육 기간 예측 프로젝트

목적 및 배경 : 한 쌍의 이미지를 입력받아 작물의 생육 기간을 예측하는 모델 개발

데이터 정보 및 학습 진행 방식 : DACON "생육 기간 예측 경진대회" 데이터

2개 작물(청경채, 적상추)에 대한 생육 기간 경과일자별 이미지 데이터 저장
- 학습 : 753개(청경채 353개, 적상추 400개)
- 테스트 : 307개(청경채 139개, 적상추 168개)


작물별 이미지 2장씩을 다양하게 조합하여 2장의 이미지간 경과일을 기준으로 학습 및 평가 진행 예정

모델 평가 기준 : RMSE(Root Mean Squared Error)

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import os
import random

import torch
from torch.utils.data import DataLoader, Dataset
from torch import optim
from torch import nn
from torchvision.transforms import ToTensor
from torchvision import transforms
from tqdm.auto import tqdm
from torchvision.models import mobilenet_v2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
# seed 고정
def seed_everything(seed):
    # 파이토치 및 넘파이, random 등 관련 모듈에 대한 seed 일괄 설정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(2048)

cpu


In [12]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

lr = 0.00005
epochs = 10
batch_size = 64
valid_batch_size = 50 ## 확인

In [13]:
# 파일 path
train_path = '/content/drive/MyDrive/ds_study/data/train_dataset/'
test_path = '/content/drive/MyDrive/ds_study/data/test_dataset/'

In [ ]:
file_name = "DAT01"

In [ ]:
file_name.split('.')[-2][-2:]

In [14]:
def extract_day(file_name):
  day = int(file_name.split('.')[-2][-2:])
  return day

In [33]:
def make_day_array(image_pathes):
  day_array = np.array([extract_day(file_name) for file_name in image_pathes])
  return day_array

In [34]:
def make_image_path_array(root_path = None):
  if root_path is None:
    bc_directories = glob('./BC/*')
    lt_directories = glob('./LT/*')
  else:
    bc_directories = glob(root_path + 'BC_RESIZE/*')
    lt_directories = glob(root_path + 'LT_RESIZE/*')
  
  bc_image_path = []  
  bc_dir = []         
  dir_idx = 0
  for bc_path in bc_directories:
    images = glob(bc_path + '/*.png')
    bc_image_path.extend(images)
    bc_dir.extend(['bc' + str(dir_idx)] * len(images))
    dir_idx += 1
  

  lt_image_path = []  
  lt_dir = []         
  dir_idx = 0
  for lt_path in lt_directories:
    images = glob(lt_path + '/*.png')
    lt_image_path.extend(images)
    lt_dir.extend(['lt' + str(dir_idx)] * len(images))
    dir_idx += 1

  return bc_image_path, lt_image_path, bc_dir, lt_dir

In [35]:
def make_dataframe(root_path=None):
  bc_image_path, lt_image_path, bc_dir, lt_dir = make_image_path_array(root_path)
  bc_day_array = make_day_array(bc_image_path)
  lt_day_array = make_day_array(lt_image_path)

  bc_df = pd.DataFrame({"file_name": bc_image_path,
                        "day" : bc_day_array,
                        "dir" : bc_dir})

  bc_df['species'] = 'bc'


  lt_df = pd.DataFrame({"file_name": lt_image_path,
                        "day" : lt_day_array,
                        "dir" : lt_dir})

  lt_df['species'] = 'lt'

  total_data_frame = pd.concat([bc_df, lt_df]).reset_index(drop=True)

  return total_data_frame

In [36]:
def make_combination(species, data_frame):
  before_file_path = []
  after_file_path = []
  time_delta = []

  for dir in tqdm(data_frame[data_frame['species'] == species]['dir'].unique()):
    for i in range(0, len(data_frame[data_frame['dir'] == dir])-1):
      for j in range(0, len(data_frame[data_frame['dir'] == dir])):
        after = data_frame[data_frame['dir'] == dir].iloc[j].reset_index(drop=True)  
        before = data_frame[data_frame['dir'] == dir].iloc[i].reset_index(drop=True)

        if int(after[1]) > int(before[1]):  
          before_file_path.append(before.iloc[0])
          after_file_path.append(after.iloc[0])
          delta = int(after.iloc[1] - before.iloc[1])
          time_delta.append(delta)

  combination_df = pd.DataFrame({
      'before_file_path' : before_file_path,
      'after_file_path' : after_file_path,
      'time_delta' : time_delta,
  })

  combination_df['species'] = species

  return combination_df

In [27]:
class KistDataset(Dataset):
  def __init__(self, combination_df, is_test=None):
    self.combination_df = combination_df
    # 텐서 변환
    self.transform = transforms.Compose([
      transforms.ToTensor()
    ])
    # 훈련용 이미지 변환
    self.transform2 = transforms.Compose([
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.RandomVerticalFlip(p=0.5),
      transforms.RandomAffine((-20, 20)),
      transforms.RandomRotation(degrees=(0, 90)),
    ])
    self.is_test = is_test
  
  def __getitem__(self, idx):
    before_image = Image.open(self.combination_df.iloc[idx]['before_file_path'])
    after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])

    before_image = self.transform(before_image)
    after_image = self.transform(after_image)
    if self.is_test:
      return before_image, after_image
    before_image = self.transform2(before_image)
    after_image = self.transform2(after_image)
    time_delta = self.combination_df.iloc[idx]['time_delta']
    return before_image, after_image, time_delta
  
  def __len__(self):
    return len(self.combination_df)

In [28]:
os.mkdir(train_path+'BC_RESIZE')
for bc in os.listdir(train_path+'BC/'):
    os.mkdir(train_path+'BC_RESIZE/'+bc)

In [29]:
os.mkdir(train_path+'LT_RESIZE')
for lt in os.listdir(train_path+'LT/'):
    os.mkdir(train_path+'LT_RESIZE/'+lt)

In [30]:
os.mkdir(test_path+'BC_RESIZE')
for dir in os.listdir(test_path+'BC/'):
    os.mkdir(test_path+'BC_RESIZE/'+dir)

In [31]:
os.mkdir(test_path+'LT_RESIZE')
for dir in os.listdir(test_path+'LT/'):
    os.mkdir(test_path+'LT_RESIZE/'+dir)

In [37]:
for bc in glob(train_path+'BC/*'):
    bc_num = bc[-5:]
    print(bc_num)
    for img in os.listdir(train_path+'BC/'+bc_num):
        img_re = Image.open(train_path+'BC/'+bc_num+'/'+img)
        img_re = img_re.resize((224, 224))
        img_re.save(train_path+'BC_RESIZE/'+bc_num+'/'+img)

BC_09
BC_06
BC_07
BC_01
BC_02
BC_08
BC_04
BC_05
BC_03


In [38]:
for lt in glob(train_path+'LT/*'):
    lt_num = lt[-5:]
    print(lt_num)
    for img in os.listdir(train_path+'LT/'+lt_num):
        img_re = Image.open(train_path+'LT/'+lt_num+'/'+img)
        img_re = img_re.resize((224, 224))
        img_re.save(train_path+'LT_RESIZE/'+lt_num+'/'+img)

LT_05
LT_03
LT_02
LT_04
LT_10
LT_07
LT_09
LT_08
LT_01
LT_06


In [39]:
for dir in os.listdir(test_path+'BC/'):
    print(dir)
    for img in os.listdir(test_path+'BC/'+ dir):
        img_re = Image.open(test_path+'BC/'+dir+'/'+img)
        img_re = img_re.resize((224, 224))
        img_re.save(test_path+'BC_RESIZE/'+dir+'/'+img)

1088
1100
1112


In [40]:
for dir in os.listdir(test_path+'LT/'):
    print(dir)
    for img in os.listdir(test_path+'LT/'+ dir):
        img_re = Image.open(test_path+'LT/'+dir+'/'+img)
        img_re = img_re.resize((224, 224))
        img_re.save(test_path+'LT_RESIZE/'+dir+'/'+img)

1003
1089
1088


In [41]:
class CompareCNN(nn.Module):
  def __init__(self):
    super(CompareCNN, self).__init__()
    self.mobile_net = mobilenet_v2(pretrained=True)
    self.fc_layer = nn.Linear(1000, 1)

  def forward(self, input):
    x = self.mobile_net(input)
    output = self.fc_layer(x)
    return output

In [42]:
class CompareNet(nn.Module):
  def __init__(self):
    super(CompareNet, self).__init__()
    self.before_net = CompareCNN()
    self.after_net = CompareCNN()

  def forward(self, before_input, after_input):
    before = self.before_net(before_input)
    after = self.after_net(after_input)
    delta = before - after
    return delta

In [52]:
total_dataframe = make_dataframe(root_path=train_path)

bc_combination = make_combination('bc', total_dataframe)
lt_combination = make_combination('lt', total_dataframe)

bc_train = bc_combination.iloc[:4500]
bc_valid = bc_combination.iloc[4500:]

lt_train = lt_combination.iloc[:4500]
lt_valid = lt_combination.iloc[4500:]

train_set = pd.concat([bc_train, lt_train])
valid_set = pd.concat([bc_valid, lt_valid])

train_dataset = KistDataset(train_set)
valid_dataset = KistDataset(valid_set)

train_data_loader = DataLoader(train_dataset,
                               batch_size = batch_size,
                               shuffle = True)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size = valid_batch_size)

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [47]:
model = CompareNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [54]:
for epoch in tqdm(range(epochs)):
  for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
    before_image = before_image.to(device)
    after_image = after_image.to(device)
    time_delta = time_delta.to(device)

    optimizer.zero_grad()
    logit = model(before_image, after_image)
    train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) / torch.LongTensor([batch_size]).squeeze(0).to(device))
    train_loss.backward()
    optimizer.step()

    if step % 15 == 0:
      print('\n=====================loss=====================')
      print(f'\n=====================EPOCH:{epoch}=====================')
      print(f'\n=====================step:{step}=====================')
      print('MAE_loss : ', train_loss.detach().cpu().numpy())
  
  valid_losses = []
  with torch.no_grad():
    for valid_before, valid_after, time_delta in tqdm(valid_data_loader):
      valid_before = valid_before.to(device)
      valid_after = valid_after.to(device)
      valid_time_delta = time_delta.to(device)

      logit = model(valid_before, valid_after)
      valid_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) / torch.LongTensor([valid_batch_size]).squeeze(0).to(device))
      valid_losses.append(valid_loss.detach().cpu())
  
  print(f'VALIDATION_LOSS MAE : {sum(valid_losses), len(valid_losses)}')
  checkpoint = {
        'model': model.state_dict(),
  }
  torch.save(checkpoint, 'checkpoint_128.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]


=====================loss=====================

=====================EPOCH:0=====================

=====================step:0=====================
MAE_loss :  1.722455

=====================loss=====================

=====================EPOCH:0=====================

=====================step:15=====================
MAE_loss :  1.7792877


0it [00:00, ?it/s]

VALIDATION_LOSS MAE : (0, 0)


...

0it [00:00, ?it/s]

VALIDATION_LOSS MAE : (0, 0)


0it [00:00, ?it/s]


=====================loss=====================

=====================EPOCH:9=====================

=====================step:0=====================
MAE_loss :  1.3186005


KeyboardInterrupt: ignored

In [ ]:
test_set = pd.read_csv('./data/test_dataset/test_data.csv')
test_set['l_root'] = test_set['before_file_path'].map(lambda x : './data/test_dataset/' + x.split('_')[1] + '_RESIZE/' + x.split('_')[2])
test_set['r_root'] = test_set['after_file_path'].map(lambda x : './data/test_dataset/' + x.split('_')[1] + '_RESIZE/' + x.split('_')[2])
test_set['l_path'] = test_set['l_root'] + '/' + test_set['before_file_path'] + '.png'
test_set['r_path'] = test_set['r_root'] + '/' + test_set['after_file_path'] + '.png'
test_dataset = KistDataset(test_set, is_test = True)
test_data_loader = DataLoader(test_dataset, batch_size = 64)

In [ ]:
test_value = []
with torch.no_grad():
  for test_before, test_after in tqdm(test_data_loader):
    test_before = test_before.to(device)
    test_after = test_after.to(device)
    logit = model(test_before, test_after)
    value = logit.squeeze(1).detach().cpu().float()

    test_value.extend(value)